In [1]:
import sqlite3 as sql
import pandas as pd

connection = sql.connect('sqlite_db_pythonsqlite.db')
cursor = connection.cursor()

In [2]:
q10 = '''
SELECT name, sum(revenue) as total_revenue 
FROM (SELECT f.name,  
       CASE WHEN b.memid > 0 THEN f.membercost * b.slots 
            ELSE f.guestcost * b.slots END AS revenue
FROM Facilities as f 
INNER JOIN Bookings as b
ON b.facid = f.facid) AS s
GROUP BY s.name
HAVING sum(revenue) < 1000
ORDER BY total_revenue;
'''

cursor.execute(q10)

print("{0:10s} \t {1:10s}".format('Facility', 'Total Revenue'))
for res in cursor.fetchall():
    print("{0:10s} \t {1:.2f}".format(*res))

Facility   	 Total Revenue
Table Tennis 	 180.00
Snooker Table 	 240.00
Pool Table 	 270.00


In [3]:
q11 = '''
SELECT firstname, surname, (SELECT firstname FROM Members m2 WHERE m2.memid = m1.recommendedby AND m1.recommendedby > 0) AS rec_first, (SELECT surname FROM Members m2 WHERE m2.memid = m1.recommendedby AND m1.recommendedby > 0) AS rec_last
FROM Members m1
WHERE memid > 0
ORDER BY surname, firstname;
'''
cursor.execute(q11)

print("{0:10s} \t {1:10s} \t {2:10s} \t {3:10s}".format('First Name', 'Last Name', 'Rec First', 'Rec Last'))
for res in cursor.fetchall():
    try:
        print("{0:10s} \t {1:10s} \t {2:10s} \t {3:10s}".format(*res))
    except:
        print("{0:10s} \t {1:10s}".format(*res))

First Name 	 Last Name  	 Rec First  	 Rec Last  
Florence   	 Bader      	 Ponder     	 Stibbons  
Anne       	 Baker      	 Ponder     	 Stibbons  
Timothy    	 Baker      	 Jemima     	 Farrell   
Tim        	 Boothe     	 Tim        	 Rownam    
Gerald     	 Butters    	 Darren     	 Smith     
Joan       	 Coplin     	 Timothy    	 Baker     
Erica      	 Crumpet    	 Tracy      	 Smith     
Nancy      	 Dare       	 Janice     	 Joplette  
David      	 Farrell   
Jemima     	 Farrell   
Matthew    	 Genting    	 Gerald     	 Butters   
John       	 Hunt       	 Millicent  	 Purview   
David      	 Jones      	 Janice     	 Joplette  
Douglas    	 Jones      	 David      	 Jones     
Janice     	 Joplette   	 Darren     	 Smith     
Anna       	 Mackenzie  	 Darren     	 Smith     
Charles    	 Owen       	 Darren     	 Smith     
David      	 Pinker     	 Jemima     	 Farrell   
Millicent  	 Purview    	 Tracy      	 Smith     
Tim        	 Rownam    
Henrietta  	 Rumney     	 Ma

In [5]:
q12 = '''
SELECT f.name, COUNT(*)
FROM Facilities f
INNER JOIN Bookings b
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY f.name;
'''
cursor.execute(q12)

print("{0:15s} \t {1:10s}".format('Facility', 'Usage'))
for res in cursor.fetchall():
    print("{0:15s} \t {1:.2f}".format(*res))

Facility        	 Usage     
Badminton Court 	 344.00
Massage Room 1  	 421.00
Massage Room 2  	 27.00
Pool Table      	 783.00
Snooker Table   	 421.00
Squash Court    	 195.00
Table Tennis    	 385.00
Tennis Court 1  	 308.00
Tennis Court 2  	 276.00


In [6]:
q13 = '''
SELECT f.name, (strftime('%Y-%m', starttime)) AS Month, COUNT(*) AS Uses
FROM Facilities f
INNER JOIN Bookings b
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY f.name, Month;
'''
cursor.execute(q13)

print("{0:15s} \t {1:10s} \t {2:10s}".format('Facility', 'Month', 'Usage'))
for res in cursor.fetchall():
    print("{0:15s} \t {1:10s} \t {2:.2f}".format(*res))

Facility        	 Month      	 Usage     
Badminton Court 	 2012-07    	 51.00
Badminton Court 	 2012-08    	 132.00
Badminton Court 	 2012-09    	 161.00
Massage Room 1  	 2012-07    	 77.00
Massage Room 1  	 2012-08    	 153.00
Massage Room 1  	 2012-09    	 191.00
Massage Room 2  	 2012-07    	 4.00
Massage Room 2  	 2012-08    	 9.00
Massage Room 2  	 2012-09    	 14.00
Pool Table      	 2012-07    	 103.00
Pool Table      	 2012-08    	 272.00
Pool Table      	 2012-09    	 408.00
Snooker Table   	 2012-07    	 68.00
Snooker Table   	 2012-08    	 154.00
Snooker Table   	 2012-09    	 199.00
Squash Court    	 2012-07    	 23.00
Squash Court    	 2012-08    	 85.00
Squash Court    	 2012-09    	 87.00
Table Tennis    	 2012-07    	 48.00
Table Tennis    	 2012-08    	 143.00
Table Tennis    	 2012-09    	 194.00
Tennis Court 1  	 2012-07    	 65.00
Tennis Court 1  	 2012-08    	 111.00
Tennis Court 1  	 2012-09    	 132.00
Tennis Court 2  	 2012-07    	 41.00
Tennis Court 2  	 2012